In [ ]:
import requests
import plotly.express as px
import pandas as pd

In [ ]:
results = requests.get("https://gis.vplanas.lt/arcgis/rest/services/Interaktyvus_zemelapis2/Seniunijos_public/MapServer/1/query?where=1%3D1&geometryType=esriGeometryEnvelope&spatialRel=esriSpatialRelIntersects&units=esriSRUnit_Foot&returnGeometry=true&returnTrueCurves=false&returnIdsOnly=false&returnCountOnly=false&returnZ=false&returnM=false&returnDistinctValues=false&returnExtentOnly=false&featureEncoding=esriDefault&f=geojson")

In [ ]:
df = pd.read_json("../ads.jsonl", lines=True)

In [ ]:
def parse_price(price_raw):
    return int(price_raw.replace(' ', '').replace('€', ''))

def parse_price_per_m2(price_raw):
    return int(price_raw.replace(' ', '').replace('(', '').replace('€/m²)', ''))

In [ ]:
df['price'] = df['price_raw'].apply(parse_price)
df['price_per_m2'] = df['price_per_m2_raw'].apply(parse_price_per_m2)

In [ ]:
projects_df = df.loc[df.project_name.notnull()]
second_hand_df = df.loc[df.project_name.isnull()]
projects_df.shape, second_hand_df.shape

In [ ]:
projects_agg_df = projects_df.groupby('district').agg({'price': ['median'], 'price_per_m2': ['median', 'count']})
second_hand_agg_df = second_hand_df.groupby('district').agg({'price': ['median'], 'price_per_m2': ['median', 'count']})

In [ ]:
projects_agg_df.columns = ['median_price', 'median_price_per_m2', 'count']
second_hand_agg_df.columns = ['median_price', 'median_price_per_m2', 'count']

In [ ]:
fig = px.choropleth(
    second_hand_agg_df.reset_index(),
    geojson=results.json(),
    color="median_price_per_m2",
    locations="district",
    featureidkey="properties.SENIUNIJA",
    projection="mercator"
)
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(title_text="Median price per m² — Second-hand apartments by district", title_y=0.95, title_x=0.5)
fig.update_layout(coloraxis_colorbar=dict(title="Median price per m² (€)"))
fig.show()